In [ ]:
from __future__ import annotations


import geopandas as gpd
import pyvista as pv
import numpy as np
import data_ingest
import time
import dtcc
import matplotlib.pyplot as plt

from dtcc_utils import dtcc_buildings_from_geodataframe,dtcc_mesh_to_pyvista_mesh
from lod1 import estimate_building_heights
from landuse import plot_landuse_categories,classify_row
from buildings_classification import assign_building_categories, CATEGORY_COLORS

from meshing import (_water_to_polydata,
                     _roads_to_polydata,
                     _landuse_categories_to_polydata,
                     )

from export_plotter import export_plotter
from IPython.display import Image, display


def show_image(png_path: str):
    display(Image(filename=png_path))


In [ ]:
place = "Thessaloniki, Greece"
domain_size = 2000.0
default_meters_per_level = 3.0

origin_point = (411894.230172,4497388.240600) # University of Macedonia
crs = "EPSG:2100"
config= data_ingest.IngestConfig(roadnetwork_type="all", #Type of road network to request
                                 use_cache= True, # Use cached files
                                 timeout= 180,  # Request timeout
                                 crs_out= crs   # Target CRS
                                 )

In [ ]:
bbox = data_ingest.square_from_point(x=origin_point[0],
                                     y=origin_point[1],
                                     input_crs=crs,
                                     side_m= domain_size,
                                    )

In [ ]:
buildings_gdf: gpd.GeoDataFrame = data_ingest.get_buildings(bbox,config)

In [ ]:
fig, ax_0 = plt.subplots(figsize=(8, 8))
buildings_gdf.plot(
    ax=ax_0,
    # facecolor="none",   # transparent fill
    edgecolor="black",  # or e.g. "#444444"
    linewidth=0.5,
    zorder=10           # draw above landuse
)
ax_0.set_title("2D Building Geometries")
plt.tight_layout()
plt.show()

In [ ]:
buildings_gdf = estimate_building_heights(buildings_gdf)

In [ ]:

ax =buildings_gdf.plot(
    column="height_final_source",   # your source column (strings)
    categorical=True,               # treat as categories
    legend=True,                    # show legend with sources
    edgecolor="black",              # optional: thin borders
    linewidth=0.1,
    figsize=(12, 8)
)
ax.set_title("2D Building Geometries height estimation source")
leg = ax.get_legend()
leg.set_bbox_to_anchor((1, 0.5))
leg._loc = 6 
plt.tight_layout()
plt.show()

In [ ]:
buildings_gdf = assign_building_categories(buildings_gdf)

In [ ]:

ax = buildings_gdf.plot(
    column="classification",   # your source column (strings)
    categorical=True,               # treat as categories
    legend=True,                    # show legend with sources
    edgecolor="black",              # optional: thin borders
    linewidth=0.1,
    figsize=(12, 8),
    cmap='tab20b'
)

ax.set_title("2D Building Geometries height estimation source")
leg = ax.get_legend()
leg.set_bbox_to_anchor((1, 0.5))
leg._loc = 6 
plt.show()

## Layer 2: Road Network

In [ ]:
roads_nodes, roads_edges, roads_graph  = data_ingest.get_roads(bbox,config)

In [ ]:
ax = buildings_gdf.plot(    
    facecolor="lightblue",   # transparent fill
    edgecolor="black",  # or e.g. "#444444"
    linewidth=0.5,
    figsize=(8,8)          
)
roads_edges.plot(ax=ax,
                 color='gray',
                 linewidth=1.0)

ax.set_title("Building Geometries and available road network")
plt.show()

## Layer 3: Land Use

In [ ]:


landuse_gdf = data_ingest.get_landuse(bounding_box=bbox, config=config)
landuse_gdf["classification"] = landuse_gdf.apply(classify_row, axis=1)

waterbodies_gdf = data_ingest.get_water(bbox,config)

In [ ]:
LANDUSE_COLOR_PALETTE = {
    "GRASS":        "#74c476",
    "FOREST":       "#238b45",
    "FARMLAND":     "#c2e699",
    "CONSTRUCTION": "#b15928",
    "INDUSTRIAL":   "#737373",
    "RESIDENTIAL":  "#ad1fb4",
    "COMMERCIAL":   "#e31a1c",
    "RELIGIOUS":    "#6a51a3",
    "MILITARY":     "#307303FF",
    "UNKNOWN":      "#bdbdbd",
}

In [ ]:
ax = plot_landuse_categories(landuse_gdf,
                             category_col= "classification",
                             palette= LANDUSE_COLOR_PALETTE,
                             opacity=0.3)

roads_edges.plot(ax=ax, 
                 color= "black",
                 linewidth=1.0)

buildings_gdf.plot(
    ax=ax,
    facecolor="none",   # transparent fill
    edgecolor="black",  # or e.g. "#444444"
    linewidth=0.5,
    zorder=10           # draw above landuse
)

waterbodies_gdf.plot(
  ax=ax,
  color="#4c77c2",
  alpha= 0.5
)

In [ ]:

buildings: dict[str,dtcc.Building] = dtcc_buildings_from_geodataframe(buildings_gdf)
buildings = dtcc.build_lod1_buildings(buildings=buildings,always_use_default_ground=True)


In [ ]:
buildings_multisurfaces: list[dtcc.MultiSurface]= [b.lod1 for _,b in buildings.items()]
meshes_list: list[dtcc.Mesh] = dtcc.builder.meshing.mesh_multisurfaces(buildings_multisurfaces,clean=False)

meshes = dict()
for building_key,mesh in zip(buildings.keys(),meshes_list):
  meshes[building_key] = mesh



In [ ]:
asset_path = "../assets/white_tower_surface.vtk"

base_asset = dtcc.load_mesh(asset_path)
white_tower_mesh = base_asset.copy()  # don’t mutate the original template

In [ ]:
target_name = "Λευκός Πύργος"

white_tower_matches = buildings_gdf[buildings_gdf["name"] == target_name]

if white_tower_matches.empty:
    print(f"No building with name '{target_name}' found")
else:
    match = white_tower_matches.iloc[0]  # take first match

uid = match["uid"]   # or whatever your ID column is called
print(f"{target_name} building has uid:", uid)

In [ ]:
building = buildings.get(uid)
if building is None:
    print(f"No Building object found for uid={uid}")
    
old_mesh = meshes.get(uid)
if old_mesh is None:
    print(f"No mesh found for uid={uid}")

In [ ]:
# # From GeoDataFrame geometry:
# centroid = match.geometry.centroid

# or, if you prefer from your own model:
centroid = building.lod0.to_polygon().centroid

print("Centroid:", centroid.x, centroid.y)

In [ ]:
white_tower_mesh.offset((centroid.x, centroid.y, 0.0))

meshes[uid] = white_tower_mesh
print(f"Replaced mesh for uid={uid} ({target_name})")

In [ ]:
plotter = pv.Plotter()
time_begin = time.time()
for i, (uid,mesh) in enumerate(meshes.items()):
    # Get classification for the i-th mesh
    label = buildings_gdf["classification"].iloc[i]

    # Pick color (fall back to "Other")
    color = CATEGORY_COLORS.get(label, CATEGORY_COLORS["Other"])

    # Convert dtcc mesh → PyVista mesh
    pv_mesh = dtcc_mesh_to_pyvista_mesh(mesh)

    # Add to plotter
    plotter.add_mesh(
        pv_mesh,
        show_edges=False,
        color=color,
        opacity=1.0,
    )
time_end = time.time()
print("Added meshes in plot in t(s)",time_end-time_begin)

In [ ]:
# Export everything in one go
paths = export_plotter(
    plotter,
    "scene.obj",
    export_static_image=True,
    export_gif=True,
    export_file=True,

)

print(paths["file"])  # scene.obj
print(paths["png"])   # scene.png
print(paths["gif"])   # scene.gif




In [ ]:
show_image(paths["png"])

In [ ]:
len(plotter.renderer.actors)

In [ ]:
roads_mesh = _roads_to_polydata(roads_edges, z=0.0)

if roads_mesh is not None:
    plotter.add_mesh(
            roads_mesh,
            color="gray",
            line_width=2,
            name="road_network",
            render_lines_as_tubes=True,
        )

In [ ]:
# Export everything in one go
paths = export_plotter(
    plotter,
    "scene_2.obj",
    export_static_image=True,
    export_gif=True,
    export_file=True,
)

print(paths["file"])  # scene.obj
print(paths["png"])   # scene.png
print(paths["gif"])   # scene.gif

In [ ]:
show_image(paths["png"])

In [ ]:
water_mesh = _water_to_polydata(waterbodies_gdf,-0.01)
if water_mesh is not None:
    plotter.add_mesh(
            water_mesh,
            color="#4c77c2",
            opacity=0.5,
            name="water_bodies",
            show_edges=False,
  
        )
    

landuse_meshes = _landuse_categories_to_polydata(
    landuse_gdf,
    categories=("GRASS", "FOREST", "FARMLAND", "RESIDENTIAL", "COMMERCIAL"),
    category_column="classification",
    base_z=-0.01,
)

for category, mesh in landuse_meshes.items():
    color = LANDUSE_COLOR_PALETTE.get(category, "#8dd3ac")
    plotter.add_mesh(
        mesh,
        color=color,
        opacity=0.35,
        name=f"landuse_{category.lower()}",
        show_edges=False,
    )

In [ ]:
# Export everything in one go
paths = export_plotter(
    plotter,
    "scene_3.obj",
    export_static_image=True,
    export_gif=True,
    export_file=True,
)

print(paths["file"])  # scene.obj
print(paths["png"])   # scene.png
print(paths["gif"])   # scene.gif

In [ ]:
show_image(paths["png"])